# Линейная регрессия


Линейные методы предполагают, что между признаками объекта и целевой переменной существует линейная зависимость, то есть:
$$ \hat{y} = w_1 x_1 + w_2 x_2 + ... + w_k x_k + b,$$
где $\hat{y}$ - целевая переменная (что мы хотим предсказать), $x_i$ - i-ый признак объекта $x$, $w_i$ - вес $i$-го признака, $b$ - bias (смещение, свободный член).

В задаче линейной регрессии $\hat{y}$ - это действительное число.

Часто для упрощения записи вводят дополнительный фиктивный признак $x_0$, который всегда равен 1, тогда bias - вес этого признака. В этом случае формула может быть записана как скалярное произведение:
$$ \hat{y} = <w, x> $$

В матричной форме формулу можно переписать следующим образом:
$$ \hat{y} = Xw,$$
$\hat{y}$ - вектор значений целевой переменной размера $n$, $X$ - матрица значений признаков объектов размера $n \times k$, w - вектор весов размера $k$. То есть в наших данных имеется $n$ объектов, каждый их которых описан $k$ признаками.

Таким образом, в матричной форме модель задаётся следующим образом:
$$ y = Xw + \epsilon$$

Важно отметить, что параметрами этой модели являются веса $w$. Когда говорят об обучении какого-либо алгоритма машинного обучения, как правило, имеют в виду настройку весов, т.е. параметров модели.  

На практике $\hat{y} $ может отличается от реальных значений, которые принимает целевая переменная $y$. Разницу между реальным значением и предсказанным, обозначим как $\epsilon$ - вектор значений случайной переменной, соответствующая случайной, непрогнозируемой ошибке модели. Ограничения, которые накладываются на эту модель:
* математическое ожидание случайных ошибок $\epsilon$ равно нулю,
* дисперсия случайных ошибок одинакова и конечна,
* случайные ошибки не скоррелированы.

Один из способов вычислить значения параметров модели, давно знаком - это наименьших квадратов, который минимизирует среднеквадратичную ошибку между реальным значением зависимой переменной и прогнозом, выданным моделью. Решение по методу наименьших квадратов дает:
$$ w = (X^TX)^{-1}X^TY $$

Загрузим необходимые библиотеки

In [304]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error, mean_squared_error
import plotly
import plotly.express as px
import plotly.graph_objs as go
theme = 'plotly_dark'
plt.style.use('ggplot')
%matplotlib inline

## Оценка результатов

Чтобы оценить качество работы алгоритма нам необходимо применяют разные метрики. Наиболее частые метрики средневадратичная и средняя абсолютная ошибки. Вычислим эти метрики на обучающей и на тестовой выборках.

 * *mean_absolute_error* - средняя абсолютная ошибка $|y_i - \hat{y}_i|$
 * *mean_squared_error* - средняя квадратичная ошибка $(y_i - \hat{y}_i)^2$

## Задание 3.1

Пример 1. Сгенерируем искусственные данные. Сначала поработаем с простейшим одномерным случаем, когда у нас значение $y$ будет зависеть только от одного значения $x$.




In [2]:
def generate_data(n_points=20):
  """
    Принимает на вход n_points точек
    Возвращает данные для обучения и теста
  """
  X = np.linspace(-5, 5, n_points)
  y = 10 * X - 7

  X_train = X[0::2].reshape(-1, 1)
  y_train = y[0::2] + np.random.randn(int(n_points/2)) * 10

  X_test = X[1::2].reshape(-1, 1)
  y_test = y[1::2] + np.random.randn(int(n_points/2)) * 10

  print(f'Generated {len(X_train)} train samples and {len(X_test)} test samples')
  return X, X_train, y_train, X_test, y_test

In [3]:
X, X_train, y_train, X_test, y_test = generate_data(100)

Generated 50 train samples and 50 test samples


In [56]:
def linear_reg1d(w, b):
    return np.vectorize(lambda x: w * x + b)

In [149]:
### Реализуйте настройку w и b с помощью рассмотренного выше метода наименьших квадратов.
### Найдите значения метрик MSE и MAE. Сравните с результатами из sklearn

X_train_f = np.hstack((X_train, np.ones_like(X_train))) #Фиктивный признак, всегда равный 1
w, b = np.linalg.inv((X_train_f.T @ X_train_f)) @ X_train_f.T @ y_train
print(f'Параметры, полученные МНК: w = {w}, b = {b}')
reg = linear_reg1d(w, b)
y_predicted = reg(X_test).ravel()
print(f'MAE: {np.mean(np.abs(y_predicted - y_test))}, MSE: {np.mean(np.square(y_predicted - y_test))}\n')

linear_regression_model = linear_model.LinearRegression().fit(X_train, y_train)
w_sl = linear_regression_model.coef_[0]
b_sl = linear_regression_model.intercept_
print(f'Параметры, полученные sklearn: w = {w_sl}, b = {b_sl}')
y_predicted_sl = linear_regression_model.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_predicted_sl)}, MSE: {mean_squared_error(y_test, y_predicted_sl)}')

Параметры, полученные МНК: w = 9.605208151634823, b = -8.267294408742593
MAE: 7.240346800815218, MSE: 88.40324514130613

Параметры, полученные sklearn: w = 9.605208151634827, b = -8.267294408742599
MAE: 7.24034680081522, MSE: 88.4032451413061


In [377]:
fig = go.Figure()
fig.add_scatter(x=X_test.ravel(), y=y_test, name='True')
fig.add_scatter(x=X_test.ravel(), y=y_predicted, name='Predicted')
fig.update_layout(hovermode='x', height=400, width=900, margin=dict(l=65, r=40, t=20, b=15), template=theme)

: 

## Задание 3.2

Пример 2. Не всегда в задаче регрессии в качестве решения выступает прямая, как в предыдущем случае. Рассмотрим ещё один пример, в котором у объектов всё ещё один признак. Но теперь мы будм брать случайную точку на синусоиде и добавлять к ней шум — таким образом получим целевую переменную, признаком в этом случае будет координата $x$.

In [256]:
def generate_wave_set(n_support=1000, n_train=25, std=0.3):
    data = {}
    # выберем некоторое количество точек из промежутка от 0 до 2*pi
    data['support'] = np.linspace(0, 2*np.pi, num=n_support)
    # для каждой посчитаем значение sin(x) + 1
    # это будет ground truth
    data['values'] = np.sin(data['support']) + 1
    # из support посемплируем некоторое количество точек с возвратом, это будут признаки
    data['x_train'] = np.sort(np.random.choice(data['support'], size=n_train, replace=True))
    # опять посчитаем sin(x) + 1 и добавим шум, получим целевую переменную
    data['y_train'] = np.sin(data['x_train']) + 1 + np.random.normal(0, std, size=data['x_train'].shape[0])
    return data

data = generate_wave_set(1000, 250)
x_train = data['support'].reshape(-1, 1)
x_test = data['x_train'].reshape(-1, 1)
y_train1 = data['values']
y_test1 = data['y_train']

In [353]:
### попробуйте реализовать настройку w и b с помощью рассмотренного выше метода наименьших квадратов.
### Найдите значения метрик MSE и MAE

x_train_f = np.hstack((x_train, np.ones_like(x_train))) #Фиктивный признак, всегда равный 1
w, b = np.linalg.inv((x_train_f.T @ x_train_f)) @ x_train_f.T @ y_train1
print(f'Параметры, полученные МНК: w = {w}, b = {b}')
reg = linear_reg1d(w, b)
y2_predicted = reg(x_test).ravel()
print(f'MAE: {np.mean(np.abs(y2_predicted - y_test1))}, MSE: {np.mean(np.square(y2_predicted - y_test1))}\n')

linear_regression_model = linear_model.LinearRegression().fit(x_train, y_train1)
w_sl = linear_regression_model.coef_[0]
b_sl = linear_regression_model.intercept_
print(f'Параметры, полученные sklearn: w = {w_sl}, b = {b_sl}')
y2_predicted_sl = linear_regression_model.predict(x_test)
print(f'MAE: {mean_absolute_error(y_test1, y2_predicted_sl)}, MSE: {mean_squared_error(y_test1, y2_predicted_sl)}')

Параметры, полученные МНК: w = -0.30305187591213784, b = 1.952065547022177
MAE: 0.44216403405568916, MSE: 0.28033738868348956

Параметры, полученные sklearn: w = -0.30305187591213884, b = 1.9520655470221808
MAE: 0.4421640340556892, MSE: 0.2803373886834896


In [375]:
fig = go.Figure()
fig.add_scatter(x=x_test.ravel(), y=y_test1, name='True')
fig.add_scatter(x=x_test.ravel(), y=y2_predicted, name='Predicted')
fig.update_layout(hovermode='x', height=400, width=900, margin=dict(l=65, r=40, t=20, b=15), template=theme)

Конечно, такое решение нас вряд ли может устроить. Нужно применить полинимиальную регрессию. Идея здесь такая. Каждый признак в исходную формулу может входить не только в первой степени, но и во второй, в третьей и так далее. То есть для случая, когда у нас только один признак:
$$ \widehat{y} = w_1 x_1 + w_2 x_1^2 + ... + w_k x_1^k + b,$$

## Задание 3.3

In [376]:
from sklearn.preprocessing import PolynomialFeatures

### Реализуйте полиномиальную регрессию. Сделайте визуализацию для полиномов разных степеней.
### Полином какой степени подходит больше других? Почему?

fig = go.Figure()
fig.add_scatter(x=x_test.ravel(), y=y_test1, name='True')
for n in range(1, 6):
    print(f'deg = {n}')
    poly = PolynomialFeatures(degree=n, include_bias=False)
    poly_x_train = poly.fit_transform(x_train.reshape(-1, 1))
    poly_x_test = poly.fit_transform(x_test.reshape(-1, 1))

    poly_reg_model = linear_model.LinearRegression()
    poly_reg_model.fit(poly_x_train, y_train1)
    y2_predicted_sl_poly = poly_reg_model.predict(poly_x_test)
    print(f'MAE: {mean_absolute_error(y_test1, y2_predicted_sl_poly)}, MSE: {mean_squared_error(y_test1, y2_predicted_sl_poly)}\n')
    fig.add_scatter(x=x_test.ravel(), y=y2_predicted_sl_poly, name=f'Pred, deg = {n}')
fig.update_layout(hovermode='x', height=400, width=900, margin=dict(l=65, r=40, t=20, b=15), template=theme)
fig.show()

deg = 1
MAE: 0.4421640340556892, MSE: 0.2803373886834896

deg = 2
MAE: 0.4421640340556892, MSE: 0.2803373886834896

deg = 3
MAE: 0.23507480574367212, MSE: 0.08794492085727194

deg = 4
MAE: 0.2350748057436737, MSE: 0.08794492085727348

deg = 5
MAE: 0.2305271387860847, MSE: 0.08637960693961429



Из графика видно, что лучше всего подходит полином 3 степени. Связано это с разложением в ряд Тейлора синуса, который используется в генерации целевой переменной

# Реальный датасет

Возьмём реальный набор данных Boston из sklearn.datasets. Этот датасет описывает средние цены на недвижимость в районах Бостона в тысячах долларов.

Примеры признаков объектов недвижимости: количество преступлений на душу населения, процент старых домов в районе, количество учеников на одного учителя и т.д. Обратите внимание на то, что данные уже оцифрованы там, где изначально признаки были качественными.

Загрузим датасет, выведем информацию

Возьмите датасет отсюда: https://www.kaggle.com/datasets/vikrishnan/boston-house-prices/

In [285]:
col_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
house_data = pd.read_csv('tables/housing.csv', sep='\s+', header=None, names=col_names)
house_data

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,22.0


## Задание 3.4

In [341]:
### оставьте в наборе данных только 7 наиболее значимых признаков
### настройте параметры линейной регрессии и сравните метрики качества (MSE и MAE) для полного датасета и усечённого

house_x = house_data.drop('MEDV', axis=1)
house_y = house_data['MEDV']
reg = linear_model.LinearRegression()
reg.fit(house_x, house_y)
print(f'Коэффициенты полного датасета:\n w = {reg.coef_}, b = {reg.intercept_}')
house_y_pred = reg.predict(house_x)
print(f'MAE: {mean_absolute_error(house_y, house_y_pred)}, MSE: {mean_squared_error(house_y, house_y_pred)}\n')

Коэффициенты полного датасета:
 w = [-1.08011358e-01  4.64204584e-02  2.05586264e-02  2.68673382e+00
 -1.77666112e+01  3.80986521e+00  6.92224640e-04 -1.47556685e+00
  3.06049479e-01 -1.23345939e-02 -9.52747232e-01  9.31168327e-03
 -5.24758378e-01], b = 36.45948838509025
MAE: 3.270862810900318, MSE: 21.8948311817292



Коэффициенты, отсортированные по значимости

In [342]:
coefs = pd.DataFrame({'var' : col_names[:-1], 'w' : np.abs(reg.coef_)}).sort_values(by='w', ascending=False)
coefs

,var,w
4,NOX,17.766611
5,RM,3.809865
3,CHAS,2.686734
7,DIS,1.475567
10,PTRATIO,0.952747
12,LSTAT,0.524758
8,RAD,0.306049
0,CRIM,0.108011
1,ZN,0.046420
2,INDUS,0.020559


In [350]:
house_x_lim = house_x[coefs['var'].head(7)]
reg_lim = linear_model.LinearRegression()
reg_lim.fit(house_x_lim, house_y)
print(f'Коэффициенты усеченного датасета:\n w = {reg_lim.coef_}, b = {reg_lim.intercept_}')
house_y_lim_pred = reg_lim.predict(house_x_lim)
print(f'MAE: {mean_absolute_error(house_y, house_y_lim_pred)}, MSE: {mean_squared_error(house_y, house_y_lim_pred)}\n')

Коэффициенты усеченного датасета:
 w = [-21.01496703   3.98766843   3.26985942  -1.15593038  -1.08792134
  -0.58115283   0.05649866], b = 40.18129933380805
MAE: 3.4715807225724404, MSE: 23.87849065746741



Можно заметить, что у усеченного датасета значения ошибок несколько выше, пусть и не очень сильно